In [17]:
from ModernBERT.src.sequence_packer import GreedyBestFitSequencePacker, SequencePacker
from datasets import load_dataset
from streaming import StreamingDataset
from transformers import AutoTokenizer
from tqdm import tqdm

In [3]:
tokenizer = AutoTokenizer.from_pretrained(f'./training/tokenizer/2048')

In [4]:
train_data = load_dataset('roneneldan/TinyStories', split='train')
val_data = load_dataset('roneneldan/TinyStories', split='validation')

In [12]:
tokenizer.add_bos_token

False

In [10]:
tokenizer("Once")

{'input_ids': [431], 'attention_mask': [1]}

In [18]:
strs = []
for i in tqdm(train_data['text'][:100000]):
    tokens = tokenizer(i, add_special_tokens=False)
    tokens['input_ids'].append(tokenizer.eos_token_id)
    strs.append([tokens])

100%|██████████| 100000/100000 [00:38<00:00, 2599.67it/s]


In [43]:
strs[0:2]

[[{'input_ids': [420, 356, 12, 259, 397, 446, 500, 364, 595, 259, 788, 307, 316, 309, 758, 14, 313, 703, 304, 282, 1031, 70, 470, 1384, 84, 266, 359, 342, 304, 787, 304, 282, 380, 287, 80, 14, 364, 444, 266, 948, 263, 788, 307, 342, 309, 365, 12, 350, 338, 460, 438, 87, 259, 1838, 303, 345, 309, 380, 312, 84, 14, 199, 199, 343, 468, 266, 309, 365, 265, 327, 12, 329, 770, 12, 335, 595, 740, 788, 307, 14, 1280, 346, 948, 304, 342, 518, 265, 438, 87, 651, 380, 312, 84, 477, 865, 365, 498, 265, 327, 12, 329, 831, 12, 364, 12, 363, 471, 948, 263, 788, 307, 265, 1304, 626, 380, 312, 84, 415, 199, 199, 52, 79, 554, 12, 362, 1654, 263, 788, 307, 265, 438, 87, 264, 263, 1838, 303, 345, 364, 371, 380, 312, 84, 14, 409, 282, 385, 1031, 70, 470, 1384, 84, 366, 448, 787, 362, 429, 380, 1394, 265, 1760, 756, 572, 14, 1451, 362, 1442, 12, 364, 857, 309, 365, 366, 380, 1394, 263, 788, 307, 265, 1304, 289, 309, 380, 312, 84, 14, 320, 896, 513, 404, 787, 362, 360, 1654, 265, 1368, 566, 14, 0], 'attentio

In [44]:
next(iter(strs))

[{'input_ids': [420, 356, 12, 259, 397, 446, 500, 364, 595, 259, 788, 307, 316, 309, 758, 14, 313, 703, 304, 282, 1031, 70, 470, 1384, 84, 266, 359, 342, 304, 787, 304, 282, 380, 287, 80, 14, 364, 444, 266, 948, 263, 788, 307, 342, 309, 365, 12, 350, 338, 460, 438, 87, 259, 1838, 303, 345, 309, 380, 312, 84, 14, 199, 199, 343, 468, 266, 309, 365, 265, 327, 12, 329, 770, 12, 335, 595, 740, 788, 307, 14, 1280, 346, 948, 304, 342, 518, 265, 438, 87, 651, 380, 312, 84, 477, 865, 365, 498, 265, 327, 12, 329, 831, 12, 364, 12, 363, 471, 948, 263, 788, 307, 265, 1304, 626, 380, 312, 84, 415, 199, 199, 52, 79, 554, 12, 362, 1654, 263, 788, 307, 265, 438, 87, 264, 263, 1838, 303, 345, 364, 371, 380, 312, 84, 14, 409, 282, 385, 1031, 70, 470, 1384, 84, 366, 448, 787, 362, 429, 380, 1394, 265, 1760, 756, 572, 14, 1451, 362, 1442, 12, 364, 857, 309, 365, 366, 380, 1394, 263, 788, 307, 265, 1304, 289, 309, 380, 312, 84, 14, 320, 896, 513, 404, 787, 362, 360, 1654, 265, 1368, 566, 14, 0], 'attention

In [19]:
out_root = f'./datasets/tinystories/2048/'
out_train = out_root + 'train/'
out_val = out_root + 'val/'

tok_train = StreamingDataset(local=out_train, shuffle=True, shuffle_seed=42, batch_size=512)


In [20]:
class IterableSequences():
    def __init__(self, seqs):
        self.seqs = seqs
        self.index = 0
    
    def __iter__(self):
        return iter(self.seqs)
    
    def __len__(self):
        return len(self.seqs)

In [21]:
seq_iter = IterableSequences(strs)

In [22]:
len(seq_iter)

100000

In [36]:

sequence_packer = GreedyBestFitSequencePacker.from_composer(
    src_iterable = seq_iter,
    micro_batch_size=1,
    batch_size=512,
    max_seq_len=512,
    pad_token_id=0,
    mask_token_id=0,
    ignore_token_id=0,
    mask_prob=0.0,
)

In [37]:
tokenizer

GPT2TokenizerFast(name_or_path='./training/tokenizer/2048', vocab_size=2048, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

In [38]:
next(iter(sequence_packer))['attention_mask'].shape

torch.Size([512, 512])

In [41]:
print(tokenizer.batch_decode(next(iter(sequence_packer))['input_ids'])[0])

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.<|endoftext|>Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.

One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that wer

In [40]:
next(iter(sequence_packer))['input_ids'].shape

torch.Size([512, 512])

In [35]:
sequence_packer

In [143]:
len(sequence_packer)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [39]:
tokenizer('upon')

{'input_ids': [1699, 303], 'attention_mask': [1, 1]}

In [52]:
for i in sequence_packer:
    print(i)
    

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



{'input_ids': tensor([[   0,  301,  282,  ...,    0,    0,    0],
        [ 343,  265,    0,  ...,    0,    0,    0],
        [1897,  503,  266,  ...,    0,    0,    0],
        ...,
        [1833,  356,   12,  ...,    0,    0,    0],
        [ 431,    0,    0,  ...,    0,    0,    0],
        [   0,  447,  259,  ...,    0,    0,    0]]), 'labels': tensor([[  42, -100, -100,  ..., -100, -100, -100],
        [-100, -100,  493,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [ 420, -100, -100,  ..., -100, -100, -100],
        [ 431,  399,  282,  ..., -100, -100, -100],
        [ 431, -100, -100,  ..., -100, -100, -100]]), 'cu_seqlens': [tensor([    0,   207,   461,   656,   865,  1073,  1335,  1511,  1740,  1980,
         2216,  2413,  2580,  2833,  3042,  3282,  3423,  3614,  3785,  3959,
         4186,  4341,  4528,  4692,  4803,  5082,  5280,  5454,  5654,  5942,
         6105,  6271,  6480,  6691,  6888,  7049,  7288,  7432,  7660,  7

In [78]:
import torch
import numpy as np
from transformers import AutoTokenizer
from typing import List, Dict, Iterable
import itertools # For simulating batching

# --- Import the packer code you provided ---
# (Assuming the code is in a file named sequence_packer.py or similar)
from sequence_packer import GreedyBestFitSequencePacker # Adjust import if needed

# 1. Setup Tokenizer and Parameters
MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Packer parameters
SRC_BATCH_SIZE = 4        # How many sentences to process from the source at a time
OUT_BATCH_SIZE = 2        # How many packed sequences per output batch
OUT_PSEQ_LEN = 64         # Target length for each packed sequence
BUFFER_SIZE = 20          # Internal buffer size (must be >= OUT_BATCH_SIZE)
PAD_TOKEN_ID = tokenizer.pad_token_id # Use tokenizer's pad ID (usually 0)
MASK_TOKEN_ID = tokenizer.mask_token_id # Use tokenizer's mask ID
IGNORE_TOKEN_ID = -100    # Standard ignore index for cross-entropy loss
MASK_PROB = 0.15          # Standard MLM probability
SUPPRESS_MASKING = False  # Perform MLM
SEED = 42

# 2. Prepare Input Data (Iterable yielding batches of dicts)
sentences = [
    "This is the first sentence.",
    "Short.",
    "This is number two, slightly longer.",
    "A third sequence for packing.",
    "Yet another one.",
    "The sixth sentence is quite a bit longer to test filling.",
    "Sentence seven.",
    "Finally, the eighth.",
    "Number nine.",
    "Tenth sentence here."
]

# Tokenize WITHOUT special tokens and padding
tokenized_data: List[Dict[str, List[int]]] = [
    {"input_ids": tokenizer.encode(s, add_special_tokens=False)}
    for s in sentences
]

print("--- Original Tokenized Data (first few) ---")
for i in range(min(5, len(tokenized_data))):
     print(f"Seq {i}: {tokenized_data[i]['input_ids']} (Len: {len(tokenized_data[i]['input_ids'])})")

# Create the source iterable that yields batches
# In a real scenario, this would often be a DataLoader
def simple_batch_generator(data: List[Dict[str, List[int]]], batch_size: int) -> Iterable[List[Dict[str, List[int]]]]:
    it = iter(data)
    while True:
        chunk = list(itertools.islice(it, batch_size))
        if not chunk:
            break
        yield chunk

src_iterable = simple_batch_generator(tokenized_data, SRC_BATCH_SIZE)

# Add dummy __len__ method if needed by the packer (it calculates based on this)
class IterableWithLen:
    def __init__(self, iterable, length):
        self._iterable = iterable
        self._length = length
    def __iter__(self):
        return iter(self._iterable)
    def __len__(self):
        return self._length

num_src_batches = (len(tokenized_data) + SRC_BATCH_SIZE - 1) // SRC_BATCH_SIZE
src_iterable_with_len = IterableWithLen(src_iterable, num_src_batches)


# 3. Instantiate the Packer
# packer = GreedyBestFitSequencePacker.from_composer(...) # Alternative using composer args

packer = GreedyBestFitSequencePacker(
    src_iterable=src_iterable_with_len,
    src_batch_size=SRC_BATCH_SIZE,
    src_max_seq_len=max(len(item['input_ids']) for item in tokenized_data), # Calculate max len
    out_batch_size=OUT_BATCH_SIZE,
    out_pseq_len=OUT_PSEQ_LEN,
    buffer_size=BUFFER_SIZE,
    pad_token_id=PAD_TOKEN_ID,
    mask_token_id=MASK_TOKEN_ID,
    ignore_token_id=IGNORE_TOKEN_ID,
    mask_prob=MASK_PROB,
    seed=SEED,
    suppress_masking=SUPPRESS_MASKING,
    # Optional: Add batch size warmup params if needed
    # batch_size_warmup_min_size=...,
    # batch_size_warmup_tokens=...,
    # world_size=...
)

# 4. Iterate Through the Packer and Inspect Output
print(f"\n--- Iterating through Packer (Output Batch Size={OUT_BATCH_SIZE}, Packed Seq Len={OUT_PSEQ_LEN}) ---")
batch_num = 0
for batch in packer:
    batch_num += 1
    print(f"\n--- Output Batch #{batch_num} ---")
    print(f"Keys: {batch.keys()}")

    input_ids = batch['input_ids']      # Shape: (OUT_BATCH_SIZE, OUT_PSEQ_LEN)
    labels = batch['labels']            # Shape: (OUT_BATCH_SIZE, OUT_PSEQ_LEN) or None
    cu_seqlens = batch['cu_seqlens']    # List[Tensor], len = OUT_BATCH_SIZE
    max_seqlen = batch['max_seqlen']    # List[int], len = OUT_BATCH_SIZE
    attention_mask = batch['attention_mask'] # Shape: (OUT_BATCH_SIZE, OUT_PSEQ_LEN)

    print(f"input_ids shape: {input_ids.shape}")
    print(f"labels shape: {labels.shape if labels is not None else 'None'}")
    print(f"attention_mask shape: {attention_mask.shape}")
    print(f"cu_seqlens length: {len(cu_seqlens)}")
    print(f"max_seqlen: {max_seqlen}")

    # Inspect the first packed sequence in the batch
    print("\nInspecting first packed sequence in the batch:")
    packed_seq_idx = 0
    print(f"  Packed Input IDs: {input_ids[packed_seq_idx].tolist()}")
    decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids[packed_seq_idx].tolist())
    print(f"  Decoded Tokens: {' '.join(decoded_tokens)}")
    print(f"  Attention Mask: {attention_mask[packed_seq_idx].tolist()}")
    if labels is not None:
        print(f"  Labels: {labels[packed_seq_idx].tolist()}")
        masked_positions = (labels[packed_seq_idx] != IGNORE_TOKEN_ID).nonzero(as_tuple=True)[0]
        print(f"  Original tokens at masked positions:")
        for pos in masked_positions:
            print(f"    Pos {pos}: {tokenizer.decode(labels[packed_seq_idx][pos])} (Input was: {tokenizer.decode(input_ids[packed_seq_idx][pos])})")

    # Understand cu_seqlens
    # cu_seqlens[i] contains [0, len_seq1, len_seq1+len_seq2, ..., total_packed_len]
    # The last element indicates the end of the last real sequence before padding.
    print(f"\n  Cumulative Sequence Lengths (cu_seqlens[{packed_seq_idx}]): {cu_seqlens[packed_seq_idx].tolist()}")
    seq_lens = cu_seqlens[packed_seq_idx][1:] - cu_seqlens[packed_seq_idx][:-1]
    print(f"  Individual sequence lengths packed: {seq_lens.tolist()}")
    num_sequences_packed = len(cu_seqlens[packed_seq_idx]) - 1
    print(f"  Number of original sequences packed: {num_sequences_packed}")
    total_token_len = cu_seqlens[packed_seq_idx][-1].item()
    print(f"  Total non-padding tokens: {total_token_len}")
    print(f"  Padding length: {OUT_PSEQ_LEN - total_token_len}")


print(f"\nFinished iterating. Consumed {packer.seqs_consumed} sequences, emitted {packer.seqs_emitted} sequences in packed batches.")

# 5. (Optional) Example with Suppressed Masking
print("\n--- Example with Suppress Masking = True ---")
# Need to recreate the iterable as the previous one was consumed
src_iterable = simple_batch_generator(tokenized_data, SRC_BATCH_SIZE)
src_iterable_with_len = IterableWithLen(src_iterable, num_src_batches)

packer_no_mask = GreedyBestFitSequencePacker(
    src_iterable=src_iterable_with_len,
    src_batch_size=SRC_BATCH_SIZE,
    src_max_seq_len=max(len(item['input_ids']) for item in tokenized_data),
    out_batch_size=OUT_BATCH_SIZE,
    out_pseq_len=OUT_PSEQ_LEN,
    buffer_size=BUFFER_SIZE,
    pad_token_id=PAD_TOKEN_ID,
    mask_token_id=MASK_TOKEN_ID,
    ignore_token_id=IGNORE_TOKEN_ID,
    mask_prob=MASK_PROB,
    seed=SEED,
    suppress_masking=True, # Set to True
)

for i, batch in enumerate(packer_no_mask):
    if i == 0: # Just show the first batch
        print(f"\n--- Output Batch #1 (No Masking) ---")
        print(f"Keys: {batch.keys()}")
        print(f"input_ids shape: {batch['input_ids'].shape}")
        print(f"labels: {batch['labels']}") # Should be None
        print(f"attention_mask shape: {batch['attention_mask'].shape}")
        print(f"\n  Packed Input IDs (first seq): {batch['input_ids'][0].tolist()}")
        decoded_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'][0].tolist())
        print(f"  Decoded Tokens (No Masking): {' '.join(decoded_tokens)}")
        print(f"  cu_seqlens (first seq): {batch['cu_seqlens'][0].tolist()}")
    else:
        break

--- Original Tokenized Data (first few) ---
Seq 0: [2023, 2003, 1996, 2034, 6251, 1012] (Len: 6)
Seq 1: [2460, 1012] (Len: 2)
Seq 2: [2023, 2003, 2193, 2048, 1010, 3621, 2936, 1012] (Len: 8)
Seq 3: [1037, 2353, 5537, 2005, 14743, 1012] (Len: 6)
Seq 4: [2664, 2178, 2028, 1012] (Len: 4)

--- Iterating through Packer (Output Batch Size=2, Packed Seq Len=64) ---

Finished iterating. Consumed 10 sequences, emitted 0 sequences in packed batches.

--- Example with Suppress Masking = True ---


In [67]:
for i in packer_no_mask:
    print(i['input_ids'])
